In [127]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter # For chunking

In [130]:
# print(dir(SimpleDirectoryReader)) # Bu sınıfın içinde hangi metodlar var listele
# help(SimpleDirectoryReader) # Bu sınıfın kullanım kılavuzunu (docstring) ekrana bas

# Loading Files
- https://developers.llamaindex.ai/python/framework/module_guides/loading/simpledirectoryreader/

- https://developers.llamaindex.ai/python/framework-api-reference/

### Reading one File and Understanding it

A data connector (aka Reader) ingest data from different data sources and data formats into a simple Document representation (text and simple metadata). For more information and different data connectors, see the [docs](https://developers.llamaindex.ai/python/framework/module_guides/loading/connector) and [modules](https://developers.llamaindex.ai/python/framework/module_guides/loading/connector/modules).

In [86]:
reader_2 = SimpleDirectoryReader(
    input_files=["../data/raw/sql/09_aggregations.pdf"]
)

# https://developers.llamaindex.ai/python/examples/data_connectors/simple_directory_reader/#full-configuration

docs = reader_2.load_data()
print(f"Loaded {len(docs)} docs") # Pdf te toplam kac sayfa varsa onu donuyor.

Loaded 31 docs


In [ ]:
type(docs)

list

In [134]:
type(docs[0])

llama_index.core.schema.Document

- Anladigim kadariyla su an tek bir pdf i okudum. Ve pdf icindeki her sayfa listenin bir elemani ve her eleman llama_index.core.schema.Document tipinde.

In [88]:
print(docs[17].get_content()) # 2. inci sayfanin icergini dondurdu 
# Slides icindeki kod ornegini yaziyor.
# docs[1].get_content() 

Groupements  GROUP BY
• Jusqu'à présent, nous avons vu des fonctions d'agrégation pour 
résumer toutes les valeurs d'une colonne ou uniquement les valeurs 
correspondant à une condition de recherche WHERE. 
• Vous pouvez utiliser la clause GROUP BY pour diviser une table en 
groupes logiques (catégories) et calculer des statistiques globales 
pour chaque groupe.
Dr R. Richardet RDB 21
SELECT
  au_id, 
  COUNT(*) AS "num_books" 
FROM title_authors 
GROUP BY au_id;


In [94]:
text = docs[17].get_content()
text

'Groupements  GROUP BY\n• Jusqu\'à présent, nous avons vu des fonctions d\'agrégation pour \nrésumer toutes les valeurs d\'une colonne ou uniquement les valeurs \ncorrespondant à une condition de recherche WHERE. \n• Vous pouvez utiliser la clause GROUP BY pour diviser une table en \ngroupes logiques (catégories) et calculer des statistiques globales \npour chaque groupe.\nDr R. Richardet RDB 21\nSELECT\n  au_id, \n  COUNT(*) AS "num_books" \nFROM title_authors \nGROUP BY au_id;'

In [135]:
temiz_text = text.replace("Dr R. Richardet RDB 21", "")
temiz_text = temiz_text.strip()
temiz_text

'Groupements  GROUP BY\n• Jusqu\'à présent, nous avons vu des fonctions d\'agrégation pour \nrésumer toutes les valeurs d\'une colonne ou uniquement les valeurs \ncorrespondant à une condition de recherche WHERE. \n• Vous pouvez utiliser la clause GROUP BY pour diviser une table en \ngroupes logiques (catégories) et calculer des statistiques globales \npour chaque groupe.\n\nSELECT\n  au_id, \n  COUNT(*) AS "num_books" \nFROM title_authors \nGROUP BY au_id;'

In [136]:
print(docs[17].get_metadata_str())
#docs[17].metadata

page_label: 18
file_name: 09_aggregations.pdf
file_path: ..\data\raw\sql\09_aggregations.pdf
file_type: application/pdf
file_size: 880600
creation_date: 2025-11-06
last_modified_date: 2025-11-22


### Reading All File ( SQL)

In [174]:
DOCUMENTS_PATH = "../data/raw/sql/"
reader = SimpleDirectoryReader(DOCUMENTS_PATH)
documents = reader.load_data()

In [175]:
print(len(documents))
print(type(documents))
documents[5].metadata

287
<class 'list'>


{'page_label': '6',
 'file_name': '01_introduction_cours_RDB.pdf',
 'file_path': 'c:\\Users\\Unal\\Documents\\source-aware-rag-tutor\\notebooks\\..\\data\\raw\\sql\\01_introduction_cours_RDB.pdf',
 'file_type': 'application/pdf',
 'file_size': 1054384,
 'creation_date': '2025-09-17',
 'last_modified_date': '2025-11-22'}

- File name:  Hangi pdf yer aldigi
- Page label: Yer aldigi pdf hangi sayfada oldugu

FileName bakarak bilginin hangi pdf ten geldigini bulabilirsin. FileName e gore gruplama yapabilirsin.


In [177]:
import re
from llama_index.core.schema import Document

# 1. Add all strings you want to remove to this list
strings_to_remove = [
    "Dr Renaud Richardet" # This specific string doesn't fit the general RDB pattern
]

# Define a regex pattern to catch variations like "Dr R. Richardet RDB 21", "Dr S. Smith RDB 1", etc.
# This pattern looks for "Dr", followed by one or more spaces,
# then typically an initial and a last name (or just a name),
# then "RDB", followed by one or more spaces and one or more digits.
dr_rdb_pattern = r"Dr\s+([A-Z]\.?\s*[A-Za-z]+\s*){1,2}RDB\s+\d+" # Made this more robust for names like "R. Richardet" or "Renaud Richardet" # Gemine kodu

updated_documents = []

for doc in documents:
    # Get the text temporarily
    clean_text = doc.text
    
    # Delete each specific string in the list one by one
    for string_to_remove in strings_to_remove:
        clean_text = clean_text.replace(string_to_remove, "")
    
    # Use regex to remove patterns like "Dr R. Richardet RDB 16"
    clean_text = re.sub(dr_rdb_pattern, "", clean_text)
    
    # Reduce multiple spaces (if any) after cleaning to a single space
    clean_text = " ".join(clean_text.split())
    
    # 2. Create a new Document object (preserving Metadata)
    new_doc = Document(text=clean_text, metadata=doc.metadata)
    updated_documents.append(new_doc)

# Update the list
documents = updated_documents

print(f"Cleaning completed. {len(documents)} documents updated.")
# For verification, let's print the beginning of the first document
print("-" * 30)
print(documents[1].text[:500])

Cleaning completed. 287 documents updated.
------------------------------
Agenda • Présentations • Organisation du cours o Plan de cours o Evaluations • Objectifs du cours • Introduction aux bases de données relationnelles (RDB) o Atelier: qualités des RDB o Définition o Composants o Couches o Cycle de vie


## Chunking

 - https://www.pinecone.io/learn/chunking-strategies/
 - https://developers.llamaindex.ai/python/framework/module_guides/loading/documents_and_nodes/usage_nodes/
 - https://github.com/run-llama/llama_cloud_services/blob/main/examples/parse/demo_advanced.ipynb

In [179]:
# https://developers.llamaindex.ai/python/framework/module_guides/loading/node_parsers/modules/#sentencesplitter
text_splitter = SentenceSplitter(
    chunk_size=1024, 
    chunk_overlap=200
)

In [180]:
nodes = text_splitter.get_nodes_from_documents(documents)

In [181]:
type(nodes)
len(nodes)


287

In [149]:
print("Nodes type = ", type(nodes))
print("Number of nodes = ", len(nodes))
print("Nodes = ", nodes)
print("Type of nodes = ", type(nodes[0]))

Nodes type =  <class 'list'>
Number of nodes =  287
Nodes =  [TextNode(id_='ee00352d-f0e4-41d5-b5f5-4eb71f001751', embedding=None, metadata={'page_label': '1', 'file_name': '01_introduction_cours_RDB.pdf', 'file_path': 'c:\\Users\\Unal\\Documents\\source-aware-rag-tutor\\notebooks\\..\\data\\raw\\sql\\01_introduction_cours_RDB.pdf', 'file_type': 'application/pdf', 'file_size': 1054384, 'creation_date': '2025-09-17', 'last_modified_date': '2025-11-22'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='bfdfef10-82ed-48d4-8830-ab3dd72e6f72', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '1', 'file_name': '01_introduction_cours_RDB.pdf', 'file_path': 'c:\\Users\\Unal\\Documents\\source-aware-rag-tutor\\notebooks\\..\\data\\raw\\sql\\01_introduction_cours_RDB.pdf', 'file_type': 'application/pdf', 'file_size': 1054384, 'creation_date': '2025-09-17', 'last_modified_date': '2025-11-22'}, hash

In [160]:
print(nodes[0].child_nodes)
nodes[0].source_node

None


RelatedNodeInfo(node_id='bfdfef10-82ed-48d4-8830-ab3dd72e6f72', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '1', 'file_name': '01_introduction_cours_RDB.pdf', 'file_path': 'c:\\Users\\Unal\\Documents\\source-aware-rag-tutor\\notebooks\\..\\data\\raw\\sql\\01_introduction_cours_RDB.pdf', 'file_type': 'application/pdf', 'file_size': 1054384, 'creation_date': '2025-09-17', 'last_modified_date': '2025-11-22'}, hash='ec4170af7439a50e67881bedbb92fe45cc32f6d4188430712cbda4c872dfdcbf')

In [193]:
print(nodes[124].text, nodes[124].source_node)

Data Definition Language (DDL) • Commandes qui créent ou changent la structure d'une table • Opérations potentiellement coûteuses et dangereuses qui demandent une bonne connaissance des RDB • Nécessitent souvent des droits spéciaux • Opérations normalement moins fréquentes, car on passe surtout du temps à créer et rechercher des données node_id='b87b58c1-1b46-4380-8d81-cbbc99799806' node_type=<ObjectType.DOCUMENT: '4'> metadata={'page_label': '17', 'file_name': '06_sql2.pdf', 'file_path': 'c:\\Users\\Unal\\Documents\\source-aware-rag-tutor\\notebooks\\..\\data\\raw\\sql\\06_sql2.pdf', 'file_type': 'application/pdf', 'file_size': 1020056, 'creation_date': '2025-09-24', 'last_modified_date': '2025-11-22'} hash='ff76c9df1edfdb8cf11d4c2c9c6a9e2cf07a4fa256b82a4b58f34e428b70f9c0'


Her nodes, her bir sayfasi temsil ediyor yani oyle bolmus su an icin ??

## Embeddings

- https://platform.openai.com/docs/guides/embeddings

In [202]:
import os
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv() # Load environment variables from .env file

# https://github.com/openai/openai-python
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

In [ ]:
# To try OpenAI
response = client.embeddings.create(
    input=nodes[0].text,
    model="text-embedding-3-small"
)

print(response.data[0].embedding)


2025-11-24 04:03:24,810 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


[0.011628035455942154, 0.020923465490341187, 0.0796351283788681, -0.031723424792289734, 0.032866403460502625, -0.0225796140730381, -0.01519692037254572, -0.005519526079297066, -0.07385026663541794, 0.023372700437903404, -0.005656566936522722, -0.04194023460149765, 0.03571217879652977, -0.07044466584920883, -0.022649593651294708, 0.050011046230793, 0.046278879046440125, 0.015803398564457893, -0.021051758900284767, 0.01804269850254059, 0.020888475701212883, -0.01700468920171261, 0.005953974090516567, -0.03860460966825485, 0.039724260568618774, 0.02799125760793686, 0.016923047602176666, 0.03965428099036217, 0.004496095702052116, -0.08215434104204178, 0.01624659262597561, -0.03930439054965973, 0.048005007207393646, 0.007306884508579969, 0.00699198292568326, 0.016001669690012932, 0.03398605436086655, 0.03452255204319954, 0.027174847200512886, 0.04310653731226921, 0.02920421212911606, 0.03032386302947998, 0.05593586340546608, -0.000989170279353857, 0.019150685518980026, 0.0030353020410984755